In [ ]:
import pandas as pd
import dask.dataframe as dd
import glob as glob
import numpy as np
from scipy.stats import kde

import matplotlib.pyplot as plt
import matplotlib.axes as plt_ax
%matplotlib inline 
from matplotlib.colors import LogNorm
plt.rc('font', size=18)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots
plt.rcParams["axes.labelsize"] = 16
plt.rcParams["axes.linewidth"] = 2.0
plt.rcParams["xtick.major.size"] = 8
plt.rcParams["xtick.minor.size"] = 6
plt.rcParams["ytick.major.size"] = 8
plt.rcParams["ytick.minor.size"] = 6
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18

In [ ]:
# input parameters for the simualtions

input_parameters={
    "minE": 10**15, #eV, 
    "maxE": 10**18, #eV,    
    "gamma":  -2.5, # slope of the spectrum
    #relation between energy and s38
    "A": 10**12, #eV
    "B": 1.2,
    #attenuation true numbers
    "alpha":0.5,
    "beta": -3.0,
    "maxTheta": 45 #degrees
}
## set the sid to generate exactly the same samples
np.random.seed(10)  
print(input_parameters)
#number of events to generate
events = 10000

import os
#filepath='/home/ioana/src/icecube/Noemi/stage/cic_att' 
filepath='/home/noemie/stage/cic_att' 
#filepath = '/Users/zhampel/icecube/analysis/CIC/stage/cic_att'
os.chdir(filepath)
%run data_functions.py

from data_functions import generate_toy_data

data = generate_toy_data(events, input_parameters)


In [ ]:
###attenuation curve
from data_functions import obtain_attenuation
nr_of_bins = 8
intensity = 500
samples = 10
import time
start_time = time.time()
intensities = np.linspace(1.5, 5., 10)
global_res = {}

for i, inte in enumerate(intensities):
    print(i, int(10**inte))
    fit_results, fitted_data = obtain_attenuation(data, nr_of_bins, int(10**inte), samples,doMCMC=True)
    global_res[inte]=(fit_results, fitted_data)
    print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
import math
groups = fitted_data
sample = fit_results
cos_ref = np.cos(math.radians(25))**2
cos2 = np.linspace(0.5, 1, 20)-cos_ref

a_mcmc,  b_mcmc,  s38_mcmc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                  zip(*np.percentile(sample, [16, 50, 84],
                                      axis=0)))

#(mid_value, +error, -error)
print("a = %f + %f - %f\n"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
print("b = %f + %f - %f\n"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
print("s_ref = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))

# Plot a subset of the samples
plt.errorbar(groups.cos2.mean()-cos_ref, groups.s125.mean(), yerr= groups.s125.std().tolist(), fmt=".k")
for a, b, f in sample[np.random.randint(len(sample), size=400)]:
    plt.plot(cos2, f * (b * cos2**2 + a * cos2 + 1), color="b", alpha=0.03)
    plt.plot(cos2, s38_mcmc[0] * (b_mcmc[0] * cos2**2 + a_mcmc[0] * cos2 + 1), color="b", lw=0.5, alpha=0.8)

plt.plot(cos2, f * (input_parameters['beta'] * cos2**2 + input_parameters['alpha'] * cos2 + 1), color="r", 
         alpha=1)
plt.errorbar(groups.cos2.mean()-cos_ref, groups.s125.mean(), yerr= groups.s125.std().tolist(), fmt=".k")
#plt.yscale('log')
plt.xlabel(r'$\cos^2(\theta) - \cos^2 (25^\circ)$', fontsize=22)
plt.ylabel(r'$S_{125}$ [VEM]', fontsize=20)

In [ ]:
import corner
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

true_vals = [input_parameters['alpha'], input_parameters['beta'], None]  
fig = corner.corner(sample, truths=true_vals, truth_color='r',
                    labels=["$a$","$b$", "$s_{ref}$"], quantiles=[0.16, 0.84],
                    show_titles=True, title_kwargs={"fontsize": 17}, color='b')

#fig.savefig("")
a_mcmc,  b_mcmc,  s38_mcmc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(sample, [16, 50, 84],
                                                axis=0)))

#(mid_value, +error, -error)
print("a = %f + %f - %f\n"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
print("b = %f + %f - %f\n"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
print("s38 = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))

# Reset x,y tick label sizes
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18

In [ ]:
#plot of a,b in fct of log(I_ref)
nr_of_bins = 10
intensity = np.arange(50,1051,100)
print(intensity)
samples = 50
a_mcmc=[]
b_mcmc=[]
sref_mcmc=[]
import time
start_time = time.time()
for i,j in enumerate(intensity):
    fit_results, fitted_data = obtain_attenuation(data, nr_of_bins, j, samples)
    a_mc,  b_mc,  s38_mc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                  zip(*np.percentile(fit_results, [16, 50, 84],
                                      axis=0)))
    
    a_mcmc.append(a_mc)
    b_mcmc.append(b_mc)
    sref_mcmc.append(s38_mc)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#plot of a,b in fct of log(I_ref)
nr_of_bins = 10
intensity = np.arange(50,1051,100)
print(intensity)
samples = 50
a_mcmc=[]
b_mcmc=[]
sref_mcmc=[]
import time
start_time = time.time()
for i,j in enumerate(intensity):
    fit_results, fitted_data = obtain_attenuation(data, nr_of_bins, j, samples)
    a_mc,  b_mc,  s38_mc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                  zip(*np.percentile(fit_results, [16, 50, 84],
                                      axis=0)))
    
    a_mcmc.append(a_mc)
    b_mcmc.append(b_mc)
    sref_mcmc.append(s38_mc)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
fig2, ax2 =plt.subplots(figsize=(10,6))
temp, groups = set_intensity(data, 10)
for name, group in groups:
    ax2.plot(np.log10(group.s125), group.I, 
             label=r'$\theta$ = {}$^\circ$'.format(math.floor(math.degrees(group.zenith.mean()))))
ax2.set_yscale("log", nonposy='clip')
plt.xlabel("$\log \ S_{125} $ [VEM]")
plt.ylabel("Intensity  [a.u.]")
plt.ylim(10, 2000)
plt.legend()


In [ ]:
fig, ax =plt.subplots()

cos_ref = np.cos(math.radians(25))**2
cos2 = np.linspace(0.5, 1, 20)-cos_ref
ar = [(0,0,0)]
br = [(0,0,0)]
sr = [(0,0,0)]

all_vals = pd.DataFrame(columns= ["a", "aer1", "aer2", "b", "ber1", "ber2", "s", "ser", "ser2", "I"])
for key in global_res:
    if int(10**key) == 3162:
        continue
    sample = global_res[key][0]
    print(sample['params'])
    params = global_res[key][0]['params']
    error = global_res[key][0]['err']
    groups = global_res[key][1]
    try:
        a_mcmc,  b_mcmc,  s38_mcmc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                      zip(*np.percentile(sample, [16, 50, 84],
                                          axis=0)))
    except:
        a_mcmc = (params[0], error[0], error[0])
        b_mcmc = (params[1], error[1], error[1])
        s38_mcmc = (params[2], error[2], error[2])
        
    all_vals = all_vals.append({"a":a_mcmc[0], "aer1":a_mcmc[1], "aer2":a_mcmc[2], "b":b_mcmc[0], "ber1":b_mcmc[1],
                    "ber2":b_mcmc[2],"s":s38_mcmc[0], "ser":s38_mcmc[1], "ser2":s38_mcmc[2],"I":key},
                   ignore_index = True)
    #print("a = %f + %f - %f"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
    #print("b = %f + %f - %f"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
    #print("s_ref = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))
    ax.errorbar(groups.cos2.mean()-cos_ref, groups.s125.mean(), yerr= groups.s125.std().tolist(),label = "", fmt=".k")
    #for a, b, f in sample[np.random.randint(len(sample), size=100)]:
    #    ax.plot(cos2, f * (b * cos2**2 + a * cos2 + 1), color="b", alpha=0.03)
    
    ax.plot(cos2, s38_mcmc[0]*(b_mcmc[0] * cos2**2 + a_mcmc[0] * cos2 + 1), lw=0.5, alpha=0.8, label="%s"%int(s38_mcmc[0]))

    
handles, labels = ax.get_legend_handles_labels()
# sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: int(t[0])))
#print(labels)
#ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)    
#plt.legend()
ax.set_yscale("log", nonposy='clip')



In [ ]:
pr_A = [5.998, 6.034, 6.081,6.139]
pr_B = [0.962, 0.948,0.936,0.923]

Fe_A = [6.069, 6.13, 6.202, 6.288]
Fe_B = [0.913, 0.900, 0.888, 0.878]
#mixed = [6.018, 6.062, 6.117,6.182]
#mixed_B = [0.938, 0.929,0.921,0.914]
#Fe_A = [6.018, 6.062, 6.117,6.182]
#Fe_B = [0.938, 0.929,0.921,0.914]

cos = [0.975, 0.925, 0.875,0.825]
energy = np.linspace(15.5, 17.5, 10)
%run data_functions.py
from utils import get_s125
param = []
param2 = []

ener = []
cos2_proxy = np.asarray(cos)**2
for ind, en in enumerate(energy):
    s_pr = 10**( (en-np.asarray(pr_A)-9) /np.asarray(pr_B) )
    s_fe = 10**( (en-np.asarray(Fe_A )-9) /np.asarray(Fe_B))

    plt.errorbar(cos2_proxy, s_pr, yerr= 0.01*s_pr, fmt=".r")
    plt.errorbar(cos2_proxy, s_fe, yerr= 0.01*s_fe, fmt=".b")
    params_pr, cov2 = sp.optimize.curve_fit(get_s125, cos2_proxy, s_pr)
    params_Fe, cov2 = sp.optimize.curve_fit(get_s125, cos2_proxy, s_fe)
    cos2 = np.linspace(0.5, 1, 20)
    param.append(params_pr)
    param2.append(params_Fe)
    
    ener.append(en)
    print(params_pr)
    print(params_Fe)
    plt.plot(cos2, params_Fe[2]*(params_Fe[1] * (cos2-cos_ref)**2 +  
                                 params_Fe[0] * (cos2-cos_ref) + 1), lw=0.5, alpha=0.8, label="Fe")
    plt.plot(cos2, params_pr[2]*(params_pr[1] * (cos2-cos_ref)**2 +  
                                 params_pr[0] * (cos2-cos_ref) + 1), lw=0.5, alpha=0.8, label="pr")
param = np.asarray(param)
param2 = np.asarray(param2)


In [ ]:
print(ener)
print(param[:, 0])
plt.subplot(211)

plt.errorbar(0.929*np.log10(all_vals.s)+6.06+9, all_vals.a, yerr= all_vals.aer1, fmt=".k")
#plt.plot(0.929*np.log10(all_vals.s)+6.06+9, np.ones(len(all_vals.s))*params_pr[0], color='r')
plt.plot(ener, param[:,0], color='r')
plt.plot(ener, param2[:,0], color='b')

#plt.plot(0.929*np.log10(all_vals.s)+6.06+9, np.ones(len(all_vals.s))*params_Fe[0], color='b')

plt.subplot(212)
plt.errorbar(0.929*np.log10(all_vals.s)+6.06+9, all_vals.b, yerr= all_vals.ber1, fmt=".k")
#plt.plot(ener, param[:,1], color='r')
#plt.plot(ener, param2[:,1], color='b')
#plt.plot(0.929*np.log10(all_vals.s)+6.06+9, np.ones(len(all_vals.s))*params_pr[1], color='r')
#plt.plot(0.929*np.log10(all_vals.s)+6.06+9, np.ones(len(all_vals.s))*params_Fe[1], color='b')



In [ ]:
font = {'family': 'serif',
        'color':  'black',
        'fontweight': 'heavy',
        'size': 20,
       }
        
a2_mcmc=np.transpose(a_mcmc) 
b2_mcmc=np.transpose(b_mcmc)
sref2_mcmc=np.log10(np.transpose(sref_mcmc))
log_errorup=1/(sref2_mcmc[0]*np.log(10))*sref2_mcmc[1]
log_errordown=1/(sref2_mcmc[0]*np.log(10))*sref2_mcmc[2]

plt.subplot(211)
plt.scatter(sref2_mcmc[0],a2_mcmc[0],color='r', alpha=0.7 )
l1 = plt.axhline(input_parameters['alpha'])
yerr_a=np.array([a2_mcmc[1],a2_mcmc[2]])
xerr_a=np.array([log_errorup,log_errordown])
plt.errorbar(sref2_mcmc[0],a2_mcmc[0],yerr_a,xerr_a, fmt=".k")
l1.set_label('true_a')
plt.xlabel("$\log S_{\text{ref}}$", fontdict=font)
plt.ylabel("a", fontdict=font)
plt.legend()
plt.subplot(212)
plt.scatter(sref2_mcmc[0],b2_mcmc[0],color='b', alpha=0.7 )
l2 = plt.axhline(input_parameters['beta'],color = 'r')
yerr_b=np.array([b2_mcmc[1],b2_mcmc[2]])
xerr_b=np.array([log_errorup,log_errordown])
plt.errorbar(sref2_mcmc[0],b2_mcmc[0],yerr_b,xerr_b, fmt=".k")
l2.set_label('true_b')
plt.xlabel("$\log S_{\text{ref}$", fontdict=font)
plt.ylabel("b", fontdict=font)
plt.legend()

plt.subplots_adjust(bottom = 0.1, top = 0.9, hspace=0.5)
plt.suptitle("MCMC constants a and b in function of $\log S_{\text{ref}}$", fontdict=font)

In [ ]:
## a and b in function of log(intensity)
log_I=np.log10(np.arange(50,1051,100))

plt.subplot(211)
plt.scatter(log_I,a2_mcmc[0],color='r', alpha=0.7 )
l1 = plt.axhline(input_parameters['alpha'])
yerr_a=np.array([a2_mcmc[1],a2_mcmc[2]])
plt.errorbar(log_I,a2_mcmc[0],yerr_a,xerr=None, fmt=".k")
l1.set_label('true_a')
plt.xlabel("$\log I_{\text{ref}}$",fontdict=font)
plt.ylabel("a", fontdict=font)
plt.legend()
plt.subplot(212)
plt.scatter(log_I,b2_mcmc[0],color='b', alpha=0.7 )
l2 = plt.axhline(input_parameters['beta'],color = 'r')
yerr_b=np.array([b2_mcmc[1],b2_mcmc[2]])
plt.errorbar(log_I,b2_mcmc[0],yerr_b,xerr=None, fmt=".k")
l2.set_label('true_b')
plt.xlabel("$\log I_{\text{ref}}$", fontdict=font)
plt.ylabel("b", fontdict=font)
plt.legend()

plt.subplots_adjust(bottom = 0.1, top = 0.9, hspace=0.5)
plt.suptitle("MCMC constants a and b in function of $\log I_{\text{ref}}$", fontdict=font)

In [ ]:
%run data_functions.py
from data_functions import set_intensity
data['s38'] = data.s125/(b_mcmc[0] * (data.cos2-cos_ref)**2 + a_mcmc[0] * (data.cos2-cos_ref) + 1)

print(data)
temp, groups = set_intensity(data, 4, 's38')
for name, group in groups:
    ax2.plot(np.log10(group.s125), group.I, 
             label='$theta$ = {}'.format(math.floor(math.degrees(group.zenith.mean()))))
ax2.set_yscale("log", nonposy='clip')
plt.xlabel("$\log S_{125}$ [VEM]")
plt.ylabel("Intensity  [a.u.]")

plt.legend()